In [22]:
import numpy as np
import pandas as pd

In [23]:
comp = pd.read_csv('data/composite-database.csv')

In [24]:
comp = comp.drop(columns = ['Unnamed: 0'])
comp.sort_values('elo', ascending = False).head(5)

,id,name,id_opp,name_opp,event_id,event_name,elo,elo_opp,height_diff,reach_diff,...,str_def_diff,td_avg_diff,td_acc_diff,td_def_diff,sub_avg_diff,wins_diff,losses_diff,momentum_diff,wl_diff_diff,outcome
8282,6506c1d34da9c013,Georges St-Pierre,2b93ebd9f5417ad2,Michael Bisping,a25b71fe5e31fa97,UFC 217: Bisping vs. St-Pierre,1280.730267,1183.346889,-2.0,1.0,...,7.0,3.11,31.0,20.0,0.8,3.0,-7.0,15.0,10.0,W
4540,6506c1d34da9c013,Georges St-Pierre,0941df56f6ac954b,Johny Hendricks,28f3c2258a1d8874,UFC 167: St-Pierre vs Hendricks,1265.910843,1173.817123,2.0,7.0,...,19.0,0.33,28.0,20.0,0.8,11.0,-6.0,15.0,17.0,W
8082,07f72a2a7591b409,Jon Jones,d967f0128c323de6,Daniel Cormier,ff9578cdbfabd323,UFC 214: Cormier vs. Jones 2,1259.237369,1161.285907,5.0,12.0,...,7.0,0.09,5.0,17.0,0.1,-3.0,0.0,-3.0,-3.0,W
4229,1f454354805b6f75,Anderson Silva,3a8176e15b9887c1,Chris Weidman,e0b74df14f52cd15,UFC 162: Silva vs Weidman,1258.335595,1105.091313,0.0,-1.0,...,11.0,-3.21,25.0,-4.0,-0.1,21.0,4.0,2.0,17.0,L
4014,6506c1d34da9c013,Georges St-Pierre,f57a8a52ca401ed9,Nick Diaz,f1b2a4365799c48b,UFC 158: St-Pierre vs Diaz,1256.974546,1040.536285,-2.0,0.0,...,11.0,2.84,41.0,23.0,0.1,7.0,-5.0,13.0,12.0,W


In [9]:
comp.loc[comp['name'] == 'Zabit Magomedsharipov']

,id,name,id_opp,name_opp,event_id,event_name,elo,elo_opp,height_diff,reach_diff,...,str_def_diff,td_avg_diff,td_acc_diff,td_def_diff,sub_avg_diff,wins_diff,losses_diff,momentum_diff,wl_diff_diff,outcome
8112,87aa9fb63d6b51a1,Zabit Magomedsharipov,bc656db347f31e19,Mike Santiago,a3244e3238541482,UFC Fight Night: Volkov vs. Struve,750.000000,750.000000,3.0,4.0,...,30.0,4.58,13.0,24.0,0.2,4.0,-3.0,7.0,7.0,W
8350,87aa9fb63d6b51a1,Zabit Magomedsharipov,0d62719b1dcf7c40,Sheymon Moraes,e7bc606d269896aa,UFC Fight Night: Bisping vs. Gastelum,766.000000,750.000000,5.0,1.0,...,-1.0,7.24,63.0,30.0,1.3,2.0,-1.0,2.0,3.0,W
8692,87aa9fb63d6b51a1,Zabit Magomedsharipov,c5af5750e8bae4f0,Kyle Bochniak,49e7e05f902479ca,UFC 223: Khabib vs. Iaquinta,781.263693,752.989595,6.0,3.0,...,6.0,6.24,43.0,15.0,1.3,2.0,-3.0,5.0,5.0,W
9094,87aa9fb63d6b51a1,Zabit Magomedsharipov,cc351e408bbb7d43,Brandon Davis,d5ae8074631762fc,UFC 228: Woodley vs. Till,795.964490,736.195845,3.0,1.0,...,2.0,7.02,30.0,20.0,1.3,3.0,-3.0,6.0,6.0,W


In [89]:
comp = pd.read_csv('data/composite-database.csv')
train = pd.read_csv('data/composite-train.csv')

In [90]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
    #Fit the model:
    model.fit(data[predictors],data[outcome])
  
    #Make predictions on training set:
    predictions = model.predict(data[predictors])
  
    #Print accuracy
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print("Accuracy : %s" % "{0:.3%}".format(accuracy))

    #Perform k-fold cross-validation with 10 folds
    kf = KFold(n_splits=10, shuffle=False)
    error = []
    for train, test in kf.split(data):
        # Filter training data
        train_predictors = (data[predictors].iloc[train,:])
    
        # The target we're using to train the algorithm.
        train_target = data[outcome].iloc[train]
    
        # Training the algorithm using the predictors and target.
        model.fit(train_predictors, train_target)
    
        #Record error from each cross-validation run
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

    #Fit the model again so that it can be refered outside the function:
    model.fit(data[predictors],data[outcome]) 

In [91]:
predictors = ['elo','elo_opp','height_diff','reach_diff','ss_min_diff','str_acc_diff', 
            'str_a_min_diff','str_def_diff','td_avg_diff','td_acc_diff','td_def_diff','sub_avg_diff', 
            'wins_diff','losses_diff','momentum_diff','wl_diff_diff']
outcome = 'outcome'
model = LogisticRegression()
classification_model(model, train, predictors, outcome)

Accuracy : 75.728%
Cross-Validation Score : 75.195%


In [92]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
classification_model(model, train, predictors, outcome)

Accuracy : 100.000%
Cross-Validation Score : 67.881%


In [93]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=30)
classification_model(model, train, predictors, outcome)

Accuracy : 99.811%
Cross-Validation Score : 76.639%


In [112]:
test = pd.read_csv('data/composite-test.csv')
results = pd.read_csv('data/test-results.csv')
test = test.iloc[::2]
test = test.reset_index()
results = results.iloc[::2]
results = results.reset_index()

In [117]:
# Logistic Regression Testing
model = LogisticRegression()
model.fit(train[predictors],train[outcome])
test[outcome] = model.predict(test[predictors])

score = 0
total = len(results)
for i in range(0, total):
    if results.loc[i, 'outcome'] == test.loc[i, 'outcome']:
        score += 1
print('Prediction rate is ' + str(round((score*100/total),2)) + '%')

Prediction rate is 89.93%


In [114]:
# Decision Tree Testing
model = DecisionTreeClassifier()
model.fit(train[predictors],train[outcome])
predictions = model.predict(train[predictors])
test[outcome] = model.predict(test[predictors])

score = 0
total = len(results)
for i in range(0, total):
    if results.loc[i, 'outcome'] == test.loc[i, 'outcome']:
        score += 1
print('Prediction rate is ' + str(round((score*100/total),2)) + '%')

Prediction rate is 72.48%


In [115]:
# Random Forest Testing
model = RandomForestClassifier()
model.fit(train[predictors],train[outcome])
predictions = model.predict(train[predictors])
test[outcome] = model.predict(test[predictors])

score = 0
total = len(results)
for i in range(0, total):
    if results.loc[i, 'outcome'] == test.loc[i, 'outcome']:
        score += 1
print('Prediction rate is ' + str(round((score*100/total),2)) + '%')

Prediction rate is 79.87%
